# <img src="https://img.icons8.com/bubbles/50/000000/mind-map.png" style="height:50px;display:inline"> EE 046211 - Technion - Deep Learning
---

## Course Project - Actually good trash statistics
---

### <img src="https://img.icons8.com/bubbles/50/000000/information.png" style="height:50px;display:inline"> Students Information
---

|Name     |Campus Email| ID  |
|---------|--------------------------------|----------|
|Alexander Balabanov| alexander.b@campus.technion.ac.il| 312775364|
|Dr. John Zoidberg| (\\\/)(;,,;)(\\\/)@campus.technion.ac.il| 3000|


### <img src="https://img.icons8.com/color/48/000000/code.png" style="height:50px;display:inline"> Get Datasets
---
The project will include 2 dataset that we are going to work with:
- [COCO](https://cocodataset.org/#home), dataset of everyday images with 330k images and 80 catagories. The data is the segmantation of the objects.
- [TACO](http://tacodataset.org/), data of trash in nature

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/MyDrive/EEDL_046211/
!git clone https://github.com/ultralytics/yolov5
!git pull
%cd ./yolov5
%pip install -qr requirements.txt
%ls -l

/content/gdrive/MyDrive/EEDL_046211
fatal: destination path 'yolov5' already exists and is not an empty directory.
fatal: not a git repository (or any parent up to mount point /content)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
/content/gdrive/MyDrive/EEDL_046211/yolov5
     |████████████████████████████████| 596 kB 4.4 MB/s 
total 14585
-rw------- 1 root root     4991 Jan 19 10:01 CONTRIBUTING.md
drwx------ 2 root root     4096 Jan 19 10:01 data/
-rw------- 1 root root    13491 Jan 19 10:01 detect.py
-rw------- 1 root root     2163 Jan 19 10:01 Dockerfile
-rw------- 1 root root    25381 Jan 19 10:01 export.py
-rw------- 1 root root     6386 Jan 19 10:01 hubconf.py
-rw------- 1 root root    35127 Jan 19 10:01 LICENSE
drwx------ 2 root root     4096 Jan 19 10:01 models/
drwx------ 2 root root     4096 Jan 19 10:04 __pycache__/
-rw------- 1 root root    15197 Jan 19 10:01 README.md
-rw------- 1 root root      926 Jan 19 10:01 requirements.txt
drwx------ 2

In [105]:
%cd /content/gdrive/MyDrive/EEDL_046211/
!git clone https://github.com/pedropro/TACO
!git pull
%cd ./TACO
%ls -l

/content/gdrive/MyDrive/EEDL_046211
Cloning into 'TACO'...
remote: Enumerating objects: 516, done.
remote: Counting objects: 100% (246/246), done.
remote: Compressing objects: 100% (160/160), done.
remote: Total 516 (delta 180), reused 149 (delta 86), pack-reused 270
Receiving objects: 100% (516/516), 70.86 MiB | 9.87 MiB/s, done.
Resolving deltas: 100% (331/331), done.
Checking out files: 100% (25/25), done.
fatal: not a git repository (or any parent up to mount point /content)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
/content/gdrive/MyDrive/EEDL_046211/TACO
total 18421
drwx------ 2 root root     4096 Jan 19 20:39 data/
-rw------- 1 root root 18845855 Jan 19 20:39 demo.ipynb
drwx------ 3 root root     4096 Jan 19 20:39 detector/
-rw------- 1 root root     1751 Jan 19 20:39 download.py
-rw------- 1 root root     1074 Jan 19 20:39 LICENSE
-rw------- 1 root root     4475 Jan 19 20:39 README.md
-rw------- 1 root root       71 Jan 19 20:39 requirements.txt

In [2]:
%cd /content/gdrive/MyDrive/EEDL_046211/TACO/
!python3 download.py

/content/gdrive/MyDrive/EEDL_046211/TACO
Note. If for any reason the connection is broken. Just call me again and I will start where I left.
Finished


In [3]:
%cd /content/gdrive/MyDrive/EEDL_046211/
%pwd

/content/gdrive/MyDrive/EEDL_046211


'/content/gdrive/MyDrive/EEDL_046211'

In [4]:
!pip install optuna


In [3]:
# imports for the tutorial
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly
import datetime
import time
import os
import csv
import json
import shutil

# pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision import transforms

# optuna
import optuna

%matplotlib inline

ModuleNotFoundError: ignored

In [6]:
%rm -fr ./datasets/taco
ann_j = "./TACO/data/annotations.json"
im_dir = "./TACO/data/"
new_dir = "./datasets/taco"
ds = json.load(open(ann_j, 'r'))
print(ds['images'][0].keys())
images = {x['id']: {'fn': x['file_name'], 'size': (x['width'], x['height'])} for x in ds['images']}
annots = [{'class' : x['category_id'], 'im_id': x['image_id'], 'bbox': x['bbox']} for x in ds['annotations']]

print("There are {} images and {} annotations".format(len(images), len(annots)))
print(annots[:3])
idx_train = sorted(np.random.choice(len(images), size=int(0.9*len(images)), replace=False))
idx_val = [x for x in range(len(images)) if x not in idx_train]

idx_type = [["train", idx_train], ['val', idx_val]]
os.mkdir(new_dir)
dir_im_ = os.path.join(new_dir, "images")
dir_lb_ = os.path.join(new_dir, "labels")
os.mkdir(dir_im_)
os.mkdir(dir_lb_)
for ds_type, idxs in idx_type:
  dir_im = os.path.join(dir_im_, ds_type)
  dir_lb = os.path.join(dir_lb_, ds_type)
  os.mkdir(dir_im)
  os.mkdir(dir_lb)  
  for idx in idx_train:
    fn_im = images[idx]['fn']
    fn_n_ = '_'.join(fn_im.split('/'))
    fn_n = os.path.join(dir_im, fn_n_)
    fn_o = os.path.join(im_dir, fn_im)

    shutil.copy(fn_o, fn_n)
    
    wd, hg = images[idx]['size']
    
    lbls = [x for x in annots if x['im_id'] == idx]
    fn_l = os.path.splitext(fn_n_)[0] + ".txt"
    fn_l = os.path.join(dir_lb, fn_l)
    f = open(fn_l, 'w')
    wr = csv.writer(f, delimiter= ' ')

    for l in lbls:
      r = [str(l['class']), 
           str(l['bbox'][0] / wd)[:8], 
           str(l['bbox'][1] / hg)[:8], 
           str(l['bbox'][2] / wd)[:8], 
           str(l['bbox'][3] / hg)[:8]]
      wr.writerow(r)
    f.close()



#Yolo format : <class> <x-center> <y_center> <w> <h> normalized [0-1]
#COCO format : <x-center> <y-center> <w> <h>

dict_keys(['id', 'width', 'height', 'file_name', 'license', 'flickr_url', 'coco_url', 'date_captured', 'flickr_640_url'])
There are 1500 images and 4784 annotations
[{'class': 6, 'im_id': 0, 'bbox': [517.0, 127.0, 447.0, 1322.0]}, {'class': 18, 'im_id': 1, 'bbox': [1.0, 457.0, 1429.0, 1519.0]}, {'class': 14, 'im_id': 1, 'bbox': [531.0, 292.0, 1006.0, 672.0]}]


In [ ]:
%rm -fr ./TACO

In [ ]:
%cd /content/gdrive/MyDrive/EEDL_046211/yolov5/
!python3 train.py --img 640 --batch 32 --epochs 8 --data ../taco.yaml --freeze 24  --name SGD_fullcat_fz24

# Grave yard

In [13]:
def Taco_ds():
  reader = csv.reader(csv_file)
  class_map = {row[0]: row[1] for row in reader}
  map_to_one_class = {c: 'Litter' for c in class_map}
  dataset = "./TACO/data/"

  # Training dataset.
  dataset_train = Taco()
  dataset_train.load_taco(dataset, 1, "train", class_map=class_map, auto_download=None)
  dataset_train.prepare()
  nr_classes = dataset_train.num_classes
  # Validation dataset
  dataset_val = Taco()
  dataset_val.load_taco(dataset, 1, "val", class_map=class_map, auto_download=None)
  dataset_val.prepare()

  return dataset_train, dataset_val


tr, vl = Taco_ds()

class TACO_DataSet(Dataset):
  def __init__(self, img_dir, ann_j, class_map, transform=None, target_transform=None):
    self.img_dir = img_dir
    self.transform = transform
    self.target_transform = target_transform
    ds = json.load(open(ann_j, 'r'))
  
  def __len__(self):
    return 0

  def __getitem__(self, idx):
    image = []
    bbox = []

    return image, bbox

creating index...
index created!
Number of images used: 1200
creating index...
index created!
Number of images used: 150


In [ ]:
class GIoU(nn.Module):
  def __init__(self):
    self.eq = 0

  def forward(pred, gt):
    return 0


def stepLrSch(opt):
  return torch.optim.lr_scheduler.StepLR(opt, 2, gamma=0.9)


def default_transform():
  return transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

In [ ]:
class HP():
  def __init__(self, name=None, epochs=16, btz=128, lr=1e-3, opt=None, crit=None, sch=None, bt_tr=None):
    self.name = datetime.datetime.now().strftime("%Y-%m-%d--%H,%M")
    self.epochs = epochs
    self.batch_size = btz
    self.lr = lr
    self.optimizer = opt if opt is not None else torch.optim.Adam
    self.criterion = crit if crit is not None else GIou()
    self.scheduler = sch if sch is not None else stepLrSch
    self.bt_transform = bt_tr if bt_tr is not None else default_transform()
  
  def get_optimizer(model):
    self.optimizer = self.optimizer(model.parameters(), lr=self.lr)
    return self.optimizer

  def get_criterion():
    return self.criterion

  def get_scheduler():
    self.scheduler = self.scheduler(self.optimizer)
    return self.scheduler

  def export():
    return None


In [ ]:
def TACO_model(classes=25, freeze=1):  
  yolo = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True, classes=classes)
  for m in [m_ for m_ in yolo.modules()][:-freeze]:
    m.requires_grad_(False)
  return yolo


In [ ]:
class DL_model():
  def __init__(self, hp=None):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.model = TACO_model().to(self.device)
    self.hp = hp if hp is not None else HP()
    self.optimizer = hp.get_optimizer(self.model)
    self.criterion = hp.get_criterion()
    self.scheduler = hp.get_scheduler()

  def evaluate():
    return 0.0

  def train():
    return None

In [99]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to /root/.cache/torch/hub/master.zip


ModuleNotFoundError: ignored

In [ ]:
dir = 'https://ultralytics.com/images/'
imgs = [dir + f for f in ('zidane.jpg', 'bus.jpg')]  # batch of images

# Inference
results = model(imgs)
results.print()  # or .show(), .save()
results.pandas().xyxy[1]

,xmin,ymin,xmax,ymax,confidence,class,name
0,223.462814,404.009460,345.418243,867.322144,0.860068,0,person
1,668.152039,401.405701,808.515930,882.466370,0.826754,0,person
2,50.385479,398.525116,242.990524,905.033630,0.808898,0,person
3,23.899151,215.100525,801.807922,779.779175,0.757313,5,bus
4,0.742684,551.577087,78.647270,891.026184,0.355743,0,person


## <img src="https://img.icons8.com/dusk/64/000000/prize.png" style="height:50px;display:inline"> Credits
---
* Icons made by <a href="https://www.flaticon.com/authors/becris" title="Becris">Becris</a> from <a href="https://www.flaticon.com/" title="Flaticon">www.flaticon.com</a>
* Icons from <a href="https://icons8.com/">Icons8.com</a> - https://icons8.com
* Datasets from <a href="https://www.kaggle.com/">Kaggle</a> - https://www.kaggle.com/